## 오픈소스SW 프로젝트_DeepFake_Detection</bn>
##### 해시값 + 딥러닝 --> Deepfake Analysis

	opencv-python: 영상 처리에 사용.
	tensorflow, keras: 딥러닝 모델 구축 및 학습.
	imagehash: 해시값 생성 및 비교.
	numpy: 수치 연산.
	scikit-image: 이미지 처리 기능 제공.


In [27]:
import os
import cv2
import imagehash
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from scipy.fftpack import fft, ifft

# 이미지 해시 생성 함수
def generate_image_hash(image_path):
    image = Image.open(image_path)
    hash_value = imagehash.phash(image)  # 퍼셉트럴 해시 사용
    return hash_value

# 해시값 비교 함수
def compare_hashes(hash1, hash2):
    return hash1 - hash2  # 해시값 간 차이를 계산 (차이가 클수록 변조 가능성 높음)

# 비디오 파일에서 프레임별 해시값 생성
def generate_video_hashes(video_path):
    video = cv2.VideoCapture(video_path)
    frame_hashes = []

    while video.isOpened():
        ret, frame = video.read()
        
        if not ret:
            print(f"Failed to read frame from {video_path}. Exiting...")
            break
        
        # 프레임이 None인지 확인
        if frame is None:
            print("Frame is None, skipping...")
            continue
        
        # 프레임의 형태 확인
        if not isinstance(frame, np.ndarray):
            print(f"Frame is not a numpy array: {type(frame)}")
            continue
        
        print(f"Read frame with shape: {frame.shape}")  # 프레임 모양 출력

        # 프레임을 이미지로 변환하고 해시값 생성
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_hash = imagehash.phash(pil_image)
        frame_hashes.append(frame_hash)

    video.release()
    return frame_hashes

# CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # 입력 크기에 맞게 조정
        self.fc2 = nn.Linear(128, 1)  # 이진 분류

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)  # Flatten
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# 비디오 폴더를 처리하는 함수
def process_video_folder(folder_path, cnn_model, threshold):
    cnn_model.eval()  # 평가 모드로 전환
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for filename in os.listdir(folder_path):
            if filename.endswith('.mp4') or filename.endswith('.avi'):  # 지원하는 비디오 형식
                video_path = os.path.join(folder_path, filename)
                print(f"Processing video: {video_path}")

                # 비디오 해시 생성
                frame_hashes = generate_video_hashes(video_path)
                if not frame_hashes:
                    print(f"No frames found in {video_path}.")
                    continue

                # 첫 번째 프레임 해시값을 기준으로 비교
                original_hash = frame_hashes[0]
                for i, frame_hash in enumerate(frame_hashes[1:], start=1):
                    hash_diff = compare_hashes(original_hash, frame_hash)
                    print(f"Frame {i} hash difference: {hash_diff}")

                    if hash_diff > threshold:  # 임계값 설정 (해시 차이 기준)
                        print(f"Potential deepfake detected in frame {i} of {filename}. Running deep learning analysis...")
                        
                        # 딥러닝 기반 탐지
                        frame_image = Image.fromarray(cv2.cvtColor(frame_hash, cv2.COLOR_RGB2BGR))
                        transform = transforms.Compose([
                            transforms.Resize((128, 128)),
                            transforms.ToTensor()
                        ])
                        frame_tensor = transform(frame_image).unsqueeze(0)  # 배치 차원 추가

                        prediction = cnn_model(frame_tensor)
                        if prediction.item() > 0.5:
                            print(f"Deepfake confirmed in frame {i} of {filename}.")
                        else:
                            print(f"Original media detected in frame {i} of {filename}.")
                    else:
                        print(f"No significant modifications detected in frame {i} of {filename}.")
            else:
                print(f"Skipping unsupported file: {filename}")

# 비디오 폴더 경로와 임계값 설정
video_folder_path = "/Users/gible/2024_2학기_오픈소스SW프로젝트/DeepFake_Project_3/test_videos"  # 비디오 폴더 경로 설정
threshold = 10  # 임계값 설정

# CNN 모델 생성 및 사용
cnn_model = SimpleCNN()
# 여기서 모델을 학습한 후에 모델을 저장하거나 불러옵니다.
# model.load_state_dict(torch.load('model.pth'))  # 모델 불러오기 예시

# 비디오 폴더 처리 실행
process_video_folder(video_folder_path, cnn_model, threshold)

Processing video: /Users/gible/2024_2학기_오픈소스SW프로젝트/DeepFake_Project_3/test_videos/qyyhuvqmyf.mp4
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 1920, 3)
Read frame with shape: (1080, 19

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'
